# Test pointcloud

In [1]:
from fdutils import tools

# import importlib

# tools = importlib.reload(tools)
# fdutils = importlib.reload(fdutils)

firedrake:WARNING OMP_NUM_THREADS is not set or is set to a value greater than 1, we suggest setting OMP_NUM_THREADS=1 to improve performance


In [2]:
from firedrake.utility_meshes import UnitSquareMesh
from firedrake.functionspace import FunctionSpace, VectorFunctionSpace
from firedrake.function import Function

from ufl.geometry import SpatialCoordinate 

def get_fun(n, degree=None, fun=None):
    degree = degree or 2
    fun = fun or (lambda x: x[0]**4 + x[1]**4)

    quadrilateral=False
    m = UnitSquareMesh(n, n, quadrilateral=quadrilateral)
    V = FunctionSpace(m, 'CG', degree)
    f = Function(V)
    x = SpatialCoordinate(m)
    f.interpolate(fun(x))
    
    return f

def get_fun_kvm(n, degree=None, fun=None):
    degree = degree or 2
    fun = fun or (lambda x: x[0]**4 + x[1]**4)

    quadrilateral=False
    m = UnitSquareMesh(n, n, quadrilateral=quadrilateral)
    V = FunctionSpace(m, 'KMV', degree)
    f = Function(V)
    x = SpatialCoordinate(m)
    f.interpolate(fun(x))
    
    return f

n = 5
fs_ = [get_fun(2**(_+1)) for _ in range(n)]
fs = [get_fun_kvm(2**(_+1)) for _ in range(n)]


In [3]:
error_handles = {}

for method in ['at', 'pc']:
    for comp in ['Cauchy', 'Reference']:
        error_handles[method + comp] = \
            tools.prepare_errornorm_handle(fs, tolerance=1e-10, eval_method=method, compare_method=comp)

errs = {}
for name, handle in error_handles.items():
    errs[name] = handle()

In [5]:
errs

{'atCauchy': [0.008594495659314444,
  0.001114545136756416,
  0.00014054731364569325,
  1.7606616315425073e-05],
 'atReference': [0.008672118609378433,
  0.0011236454233559652,
  0.00014164994703833362,
  1.7606616315425073e-05],
 'pcCauchy': [0.008594495659314453,
  0.0011145451367564141,
  0.00014054731364569612,
  1.7606616315425382e-05],
 'pcReference': [0.008672118609378431,
  0.0011236454233559672,
  0.0001416499470383321,
  1.7606616315425382e-05]}